In [2]:
#import library
import psycopg2
import pandas as pd
import csv
import math

In [141]:
#input database
host="localhost"
database="python"
user="postgres"
password="Pplatinum01"

In [142]:
#connect ke database
conn = psycopg2.connect(host=host, database=database, user=user, password=password)
cur = conn.cursor()

In [1]:
cur.execute("""
SELECT table_schema,table_name
FROM information_schema.tables
ORDER BY table_schema,table_name;""")
cur.fetchall()

In [3]:
#load data dari local ke python
campaign = pd.read_excel('E:/iykra/tugas/tugas query python/Downloads/campaign.xlsx')
# campaign_details = pd.read_excel('E:/iykra/tugas/tugas query python/Downloads/campaign_details.xlsx')
# delivorder = pd.read_csv('E:/iykra/tugas/tugas query python/Downloads/delivorder.csv')
# delivorder_item = pd.read_csv('E:/iykra/tugas/tugas query python/Downloads/delivorder_item.csv')
# distance = pd.read_excel('E:/iykra/tugas/tugas query python/Downloads/distance.xlsx')
# geographic = pd.read_excel('E:/iykra/tugas/tugas query python/Downloads/geographic.xlsx')
# plants = pd.read_excel('E:/iykra/tugas/tugas query python/Downloads/plants.xlsx')
# produk = pd.read_excel('E:/iykra/tugas/tugas query python/Downloads/produk.xlsx')
# sales_order2 = pd.read_excel('E:/iykra/tugas/tugas query python/Downloads/sales_order2.xlsx')
# search_keyword = pd.read_excel('E:/iykra/tugas/tugas query python/Downloads/search_keyword.xlsx')
# user_location = pd.read_excel('E:/iykra/tugas/tugas query python/Downloads/user_location.xlsx')


In [144]:
#fungsi untuk memotong table
def truncate_table(table):
    table = table.truncate(after =999)
    return table   

In [145]:
#memotong data sampai ke row seribu.
delivorder = truncate_table(delivorder)
delivorder_item = truncate_table(delivorder_item)
sales_order2 = truncate_table(sales_order2)

In [169]:
##fungsi membuat table create table
def create_tables(conn,cur):
    """ create tables in the PostgreSQL database"""
    commands = (
        """
        CREATE TABLE public.campaign (
                campaign VARCHAR PRIMARY KEY,
                state VARCHAR,
                type VARCHAR,
                currency VARCHAR,
                clicks INTEGER,
                impressions INTEGER,
                ctr INTEGER,
                avg_cpc FLOAT,
                cost FLOAT,
                conversions FLOAT,
                view_through_conv INTEGER,
                cost_per_conv FLOAT,
                conv_rate FLOAT)
        """,
        """
        CREATE TABLE public.campaign_details (
                campaign VARCHAR PRIMARY KEY,
                state VARCHAR,
                type VARCHAR,
                subtype VARCHAR,
                labels VARCHAR,
                bid_strategy VARCHAR,
                bid_strategy_type VARCHAR,
                sitelinks_active INTEGER,
                sitelinks_disapproved INTEGER,
                phone_active INTEGER,
                phone_disapproved INTEGER,
                apps_active INTEGER,
                apps_disapproved INTEGER,
                apps_level VARCHAR,
                desktop_bid_adj VARCHAR,
                mobile_bid_adj VARCHAR,
                tablet_bid_adj VARCHAR,
                currency VARCHAR,
                clicks INTEGER,
                impressions INTEGER,
                ctr INTEGER,
                avg_cpc FLOAT,
                cost FLOAT,
                impr_abs_top FLOAT,
                impr_top FLOAT,
                conversions FLOAT,
                view_through_conv INTEGER,
                cost_per_conv FLOAT,
                conv_rate FLOAT)
        """,
        """
        CREATE TABLE public.delivorder (
                do_id INTEGER,
                do_tgl VARCHAR,
                do_org_id INTEGER,
                do_so_id INTEGER,
                do_tm_id FLOAT,
                do_nomor VARCHAR,
                do_driver_id FLOAT,
                do_status_verifikasi INTEGER,
                do_is_invoice INTEGER,
                do_is_tolak INTEGER,
                do_tolak_ket FLOAT,
                do_tgl_tolak FLOAT,
                do_jam_berangkat VARCHAR,
                do_ket FLOAT,
                do_bag_pengiriman_id FLOAT,
                do_createtime VARCHAR,
                do_tgl_terima VARCHAR,
                do_is_jurnal INTEGER,
                do_jam_terima VARCHAR,
                do_jam_datang_diproyek VARCHAR,
                do_jam_bongkar VARCHAR,
                do_tgl_datang_diproyek VARCHAR,
                do_tgl_bongkar VARCHAR,
                do_is_invoice_plant_utama INTEGER,
                do_stat_id FLOAT,
                do_tgl_pulang VARCHAR,
                do_jam_pulang VARCHAR,
                do_jarak_km FLOAT,
                do_ba_penolakan VARCHAR,
                do_pembuat_penolakan FLOAT,
                do_penyebab_penolakan VARCHAR,
                do_penyetuju_penolakan FLOAT,
                do_tld_id FLOAT,
                do_hari_ba_penolakan VARCHAR,
                do_tgl_ba_penolakan VARCHAR,
                do_barcode FLOAT,
                do_alasan_berkas_belum_terima FLOAT,
                do_is_berkas_diterima FLOAT,
                do_tgl_berkas_diterima VARCHAR,
                do_tempat_cor VARCHAR,
                do_lokasi_terima FLOAT,
                PRIMARY KEY (do_id,do_nomor))
        """,
        """
        CREATE TABLE public.delivorder_item(
                deliv_order SERIAL PRIMARY KEY,
                doi_id INTEGER,
                doi_do_id INTEGER,
                doi_soi_id INTEGER,
                doi_qty_kirim FLOAT,
                doi_qty_terima FLOAT,
                ket VARCHAR(255))
        """,
        """
        CREATE TABLE public.distance(
                from_location VARCHAR(255) PRIMARY KEY, 
                currency VARCHAR(255),
                clicks INTEGER,
                impressions INTEGER,
                ctr FLOAT,
                avg_cpc FLOAT,
                cost FLOAT,
                conversions FLOAT,
                view_through_conv INTEGER,
                cost_per_conv FLOAT,
                conv_rate INTEGER)
        """,
        """
        CREATE TABLE public.geographic (
                geo_id SERIAL PRIMARY KEY,
                country_territory VARCHAR(255),
                region VARCHAR(255),
                metro_area VARCHAR(255),
                city VARCHAR(255),
                most_targeted_location VARCHAR(255),
                location_type VARCHAR(255),
                currency VARCHAR(255),
                clicks INTEGER,
                impressions INTEGER,
                ctr FLOAT,
                avg_cpc FLOAT,
                cost FLOAT,
                impr_abs_top FLOAT,
                impr_top FLOAT,
                conversions FLOAT,
                view_through_conv INTEGER,
                cost_per_conv FLOAT,
                conv_rate FLOAT)
        """,
        """
        CREATE TABLE public.plants (
                plant VARCHAR PRIMARY KEY,
                batching_tipe VARCHAR,
                batching_jml INTEGER,
                batching_kapasitas INTEGER,
                status VARCHAR,
                jml_truk FLOAT)
        """,
        """
        CREATE TABLE public.produk (
                no INTEGER PRIMARY KEY,
                mutu VARCHAR(255),
                spesifikasi VARCHAR(255),
                slump VARCHAR(255),
                spek_tambahan VARCHAR(255))
        """,
        """
        CREATE TABLE public.sales_order2 (
                tanggal_pemesanan TIMESTAMP,
                nomor_kontrak_retail VARCHAR(255) PRIMARY KEY,
                pelanggan VARCHAR(255),
                jenis VARCHAR(255),
                nama_proyek VARCHAR(255),
                product VARCHAR(255),
                area VARCHAR(255),
                nilai INTEGER,
                status VARCHAR(255))
        """,
        """
        CREATE TABLE public.search_keyword (
                id_keyword SERIAL PRIMARY KEY,
                keyword VARCHAR,
                keyword_status VARCHAR,
                keyword_match_type VARCHAR,
                campaign VARCHAR,
                ad_group VARCHAR,
                keyword_max_cpc INTEGER,
                currency VARCHAR,
                clicks INTEGER,
                impressions INTEGER,
                ctr FLOAT,
                avg_cpc FLOAT,
                cost INTEGER,
                impr_abs_top FLOAT,
                impr_top FLOAT,
                conversions FLOAT,
                view_through_conv INTEGER,
                cost_conv FLOAT,
                conv_rate FLOAT,
                avg_cpm FLOAT,
                FOREIGN KEY (campaign) REFERENCES campaign (campaign))
        """,
        """
        CREATE TABLE public.user_location (
                user_loc_iD SERIAL PRIMARY KEY,
                country VARCHAR,
                region VARCHAR,
                metro_area VARCHAR,
                city VARCHAR,
                most_targeted_location VARCHAR,
                location_type VARCHAR,
                currency VARCHAR,
                clicks INTEGER,
                impressions INTEGER,
                ctr FLOAT,
                avg_cpc FLOAT,
                cost FLOAT,
                impr_abs_top FLOAT,
                impr_top FLOAT,
                conversions FLOAT,
                view_through_conv INTEGER,
                cost_conv FLOAT,
                conv_rate FLOAT)
        """,
    )
    for command in commands:
        cur.execute(command)
    conn.commit() #untuk mengsave perubahan
    cur.close() #untuk ngeclose cursor, ada juga close connection: conn.close()
    

In [170]:
##create tables
create_tables(conn,cur)

In [149]:
#memberikan nama kepada data frame
campaign.name = "campaign"
campaign_details.name = "campaign_details"
delivorder.name = "delivorder"
delivorder_item.name = "delivorder_item"
distance.name = "distance"
geographic.name = "geographic"
plants.name = "plants"
produk.name = "produk"
sales_order2.name = "sales_order2"
search_keyword.name = "search_keyword"
user_location.name = "user_location"

#membuat list dengan elemen dataframe
list_table=[campaign,
            campaign_details,
            delivorder,
            delivorder_item, 
            distance, 
            geographic,
            plants,
            produk,
            sales_order2,
            search_keyword,
            user_location]

In [150]:
#fungsi cek primary key dengan keunikan nilai pada kolom
def cek_unik(df):
    kolom = df.columns
    for col in kolom:
        if len(df[col].unique())==len(df[col]):
            print (col)

In [151]:
#run fungsi cek primary key
for tabel in list_table:
    print(("----------------------- %s ------------------------") %(tabel.name))
    cek_unik(tabel)

----------------------- campaign ------------------------
campaign
----------------------- campaign_details ------------------------
campaign
----------------------- delivorder ------------------------
do_id
do_nomor
----------------------- delivorder_item ------------------------
doi_id
doi_do_id
----------------------- distance ------------------------
from_location
clicks
impressions
avg_cpc
cost
conversions
cost_per_conv
----------------------- geographic ------------------------
----------------------- plants ------------------------
plant
----------------------- produk ------------------------
no
----------------------- sales_order2 ------------------------
nomor_kontrak_retail
----------------------- search_keyword ------------------------
----------------------- user_location ------------------------


In [152]:
####menentukan foreign key
#membuat dictionary primary key

primary ={0:{'table': 'campaign',
             'key': 'campaign'},
          1:{'table': 'campaign_details',
             'key': 'campaign'},
          2:{'table': 'delivorder',
             'key': 'do_id'},
          3:{'table': 'distance',
             'key': 'from_location'},
          4:{'table': 'plants',
             'key': 'plant'},
          5:{'table': 'produk',
             'key': 'no'},
          6:{'table': 'sales_order2',
             'key': 'nomor_kontrak_retail'},
          }
#membuat dataframe primary key
primary_key = pd.DataFrame.from_dict(primary, orient='index')
primary_key

,table,key
0,campaign,campaign
1,campaign_details,campaign
2,delivorder,do_id
3,distance,from_location
4,plants,plant
5,produk,no
6,sales_order2,nomor_kontrak_retail


In [153]:
# fungsi cek foreign key dalam tabel
def cek_fk(table, primary_key):
    for col in table.columns:
        for i in range(len(primary_key['key'])):
            if table.name != primary_key['table'][i]: 
                if col == primary_key['key'][i]:
                    print (("kolom %s, FK dengan tabel %s") %(col,primary_key['table'][i]))

In [154]:
#run cek foreign key
for table in list_table:
    print (("-----------%s-----------") %(table.name))
    cek_fk(table, primary_key)

-----------campaign-----------
kolom campaign, FK dengan tabel campaign_details
-----------campaign_details-----------
kolom campaign, FK dengan tabel campaign
-----------delivorder-----------
-----------delivorder_item-----------
-----------distance-----------
-----------geographic-----------
-----------plants-----------
-----------produk-----------
-----------sales_order2-----------
-----------search_keyword-----------
kolom campaign, FK dengan tabel campaign
kolom campaign, FK dengan tabel campaign_details
-----------user_location-----------


In [155]:
#fungsi query insert ke postgresql
def insert_value(table):
    cols = ",".join([str(i) for i in table.columns.tolist()])
    for i,row in table.iterrows():
        sql2 = "INSERT INTO "+ table.name +" (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cur.execute(sql2, tuple(row))
        conn.commit()


In [173]:
#run program untuk semua table dalam list_table
for table in list_table:
    insert_value(table)

In [172]:
#connect ke database
cur.close()
conn = psycopg2.connect(host=host, database=database, user=user, password=password)
cur = conn.cursor()

In [7]:
campaign.iterrows(1)

TypeError: iterrows() takes 1 positional argument but 2 were given